In [ ]:
from PIL import Image
from pylab import *
import cv2
from numpy import *
from numpy.ma import array
from scipy.ndimage import filters
from PIL import Image 

In [ ]:
""" 使用归一化的互相关计算视差图像 """

def plane_sweep_ncc(im_l,im_r,start,steps,wid):
    m,n = im_l.shape
    mean_l = zeros((m,n))
    mean_r = zeros((m,n))
    s = zeros((m,n))
    s_l = zeros((m,n))
    s_r = zeros((m,n))
    dmaps = zeros((m,n,steps))      # 保存深度平面的数组
    filters.uniform_filter(im_l,wid,mean_l)     # 计算图像块的平均值
    filters.uniform_filter(im_r,wid,mean_r)
    norm_l = im_l - mean_l
    norm_r = im_r - mean_r
    
    # 尝试不同的视差
    for displ in range(steps):
        filters.uniform_filter(np.roll(norm_l, -displ - start) * norm_r, wid, s)
        filters.uniform_filter(np.roll(norm_l, -displ - start) * np.roll(norm_l, -displ - start), wid, s_l)
        filters.uniform_filter(norm_r*norm_r,wid,s_r)
        dmaps[:,:,displ] = s / sqrt(s_l * s_r)
        
    return np.argmax(dmaps, axis=2)     # 为每个像素选取最佳深度

In [ ]:
#使用带有高斯加权周边的归一化互相关计算视差图像

def plane_sweep_gauss(im_l,im_r,start,steps,wid):
    m,n = im_l.shape
    mean_l = zeros((m,n))
    mean_r = zeros((m,n))
    s = zeros((m,n))
    s_l = zeros((m,n))
    s_r = zeros((m,n))
    dmaps = zeros((m,n,steps))
    filters.gaussian_filter(im_l,wid,0,mean_l)
    filters.gaussian_filter(im_r,wid,0,mean_r)
    norm_l = im_l - mean_l
    norm_r = im_r - mean_r
    
    # 尝试不同的视差
    for displ in range(steps):
        filters.gaussian_filter(np.roll(norm_l, -displ - start) * norm_r, wid, 0, s)
        filters.gaussian_filter(np.roll(norm_l, -displ - start) * np.roll(norm_l, -displ - start), wid, 0, s_l)
        filters.gaussian_filter(norm_r*norm_r,wid,0,s_r)
        dmaps[:,:,displ] = s / np.sqrt(s_l * s_r)
    
    return np.argmax(dmaps, axis=2)     # 为每个像素选取最佳深度

In [ ]:
im_l = array(Image.open('data/1.png').convert('L'), 'f')
im_r = array(Image.open('data/2.png').convert('L'),'f')
# 开始偏移，并设置步长
steps = 12
start = 4
# ncc 的宽度
wid = 2
res = plane_sweep_gauss(im_l,im_r,start,steps,wid)



#保存视差图
outputImg = Image.fromarray(res*255.)
outputImg = outputImg.convert('L')
outputImg.save('depth14.jpg')  
outputImg.show()



In [ ]:
import matplotlib.pyplot as plt
import cv2
imageA = cv2.imread('data/1.png')
imageB = cv2.imread('data/2.png')
img = cv2.imread('depth6.jpg')
plt.figure(figsize=(120, 120))
plt.subplot(1,3,1,xticks = [],yticks=[])
plt.title("left",size=220)
plt.imshow(imageA)
plt.subplot(1,3,2,xticks = [],yticks=[])
plt.title("right",size=220)
plt.imshow(imageB)
plt.subplot(1,3,3,xticks = [],yticks=[])
plt.title("depth(width:6)",size=220)
plt.imshow(img)
plt.show()